## API情報を取得





In [1]:
import requests
import json
import pandas as pd
import boto3
import datetime
import os
from io import StringIO


# APIのひな型
api_key = os.environ['steamapikey']
user_key = os.environ['steamuserkey']
url= "http://api.steampowered.com/IPlayerService/GetOwnedGames/v0001/?key={}&steamid={}&format=json&include_appinfo=1".format(api_key,user_key)


# S3のクレデンシャル情報は環境設定で指定。
S3_accesskey = os.environ['s3accesskey']
S3_secretkey = os.environ['s3secretkey']


# S3のバケットを指定
Bucket_profile = 'buc-steamapi'



In [2]:
r = requests.get(url)
# 結果はJSON形式なのでデコードする
data = json.loads(r.text) 


df = pd.DataFrame(data['response']['games'])  

df = df[["appid","name","playtime_forever"]]

df["day"] = datetime.date.today()

In [3]:
df.head()

,appid,name,playtime_forever,day
0,70,Half-Life,2,2019-05-23
1,220,Half-Life 2,1,2019-05-23
2,340,Half-Life 2: Lost Coast,0,2019-05-23
3,320,Half-Life 2: Deathmatch,0,2019-05-23
4,360,Half-Life Deathmatch: Source,0,2019-05-23


In [4]:
def  upload_df_to_s3_as_csv(df):
    s3 = boto3.resource('s3',
                      aws_access_key_id=S3_accesskey,
                      aws_secret_access_key=S3_secretkey,
                      region_name='ap-northeast-1')

    
    csv_buffer = StringIO()
    df.to_csv(csv_buffer)
    s3.Object(Bucket_profile, 'dailydata/df_{}.csv'.format(datetime.date.today())).put(Body=csv_buffer.getvalue())
    

In [5]:
upload_df_to_s3_as_csv(df)